HOLA QUE TAL

In [ ]:
!pip install paho-mqtt==1.5.1
!pip install openpyxl
!pip install pyodbc
#gola que tal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.1-py3-none-any.whl size=61549 sha256=b71c5ca7152e06b7d93e303f09c026b069e1754bb3a60b07b27e7524605a7c9b
  Stored in directory: /root/.cache/pip/wheels/b6/72/f9/44ca415a44f9cc7468fe4c59e6282938e504fadec09641fa63
Successfully built paho-mqtt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 5.8 MB/s eta 0:00:00


# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Historical Data of Humanities Library

Two Excel Files:

Before Solar Panels

After Solar Panels

In [ ]:
columns_to_read = ['Date', 'Hour', 'ET_Humanitats (General ET) [kWh]']
data = pd.read_excel('/content/sample_data/Energia activa - 15min_01-01-2018_10-04-2024.xls',skiprows=14, usecols=columns_to_read)

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Energia activa - 15min_01-01-2018_10-04-2024.xls'

In [ ]:
"""
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

#train_data.to_csv('/content/drive/MyDrive/Synthesis Project/train_data.csv', index=False)
#test_data.to_csv('/content/drive/MyDrive/Synthesis Project/test_data.csv', index=False)
"""


# Sensors Real Time Data

In [ ]:
"""import paho.mqtt.client as mqtt
import json

APPEUI = "q4-1003-7456"
APPID  = "sensors-openlab@ttn"
PSW    = 'NNSXS.AKU22YDKMFUNHGKYVJMMQVDLOBYMQBLAACLENZA.2KOWOPCVUXWCLN6SVP4LRZUYKBKMP7XDWB7OJHQENFMTVF4JSLFA'

# Call back functions
def on_message(mqttc, userdata, msg):
    print("Message received")
    print("Topic: " + msg.topic)
    #print("Payload: " + msg.payload.decode())
    # Check if the message is from the specific device
    if msg.topic == 'v3/' + APPID + '/devices/' + APPEUI + '/up':
        on_message2(mqttc, userdata, msg)

# Gives connection message
def on_connect(mqttc, mosq, obj, rc):
    print("Connected with result code:" + str(rc))
    # Subscribe to the specific device's uplink messages
    mqttc.subscribe('v3/' + APPID + '/devices/' + APPEUI + '/up')
    print("Subscribed to topic: v3/" + APPID + "/devices/" + APPEUI + "/up")

def on_message2(mqttc, userdata, msg):
    payload = json.loads(msg.payload.decode())
    decoded_payload = payload["uplink_message"]["decoded_payload"]

    # Extracting sensor data
    activity = decoded_payload["activity"]
    co2 = decoded_payload["co2"]
    humidity = decoded_payload["humidity"]
    illumination = decoded_payload["illumination"]
    infrared = decoded_payload["infrared"]
    infrared_and_visible = decoded_payload["infrared_and_visible"]
    pressure = decoded_payload["pressure"]
    temperature = decoded_payload["temperature"]
    tvoc = decoded_payload["tvoc"]

    # Print sensor data
    print("Sensor Data:")
    print("Activity:", activity)
    print("CO2:", co2)
    print("Humidity:", humidity)
    print("Illumination:", illumination)
    print("Infrared:", infrared)
    print("Infrared and Visible:", infrared_and_visible)
    print("Pressure:", pressure)
    print("Temperature:", temperature)
    print("TVOC:", tvoc)

mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.username_pw_set(APPID, PSW)
mqttc.connect("eu1.cloud.thethings.network", 1883, 60)

mqttc.loop_forever()"""


In [ ]:
import paho.mqtt.client as mqtt
import json
from datetime import datetime
import pytz
from openpyxl import Workbook
from openpyxl import load_workbook
import os
import pyodbc


def save_data_to_excel(data, filename):
    try:
       # Create a new Excel workbook if the file doesn't exist
        if not os.path.exists(filename):
            wb = Workbook()
            ws = wb.active

            # Add headers
            headers = ["Time", "Sensor", "CO2", "Humidity", "Illumination", "Temperature", "TVOC"]
            ws.append(headers)
        else:
            # Load existing workbook
            wb = load_workbook(filename)
            ws = wb.active

        # Add data
        for row in data:
          if row not in ws.values:
              ws.append(row)

        # Save the Excel file
        wb.save(filename)
        print(f"Data saved to '{filename}' successfully.")

    except Exception as e:
        print("Error saving data to Excel file:", e)

collected_data = []


APPID = "sensors-openlab@ttn"
PSW = 'NNSXS.AKU22YDKMFUNHGKYVJMMQVDLOBYMQBLAACLENZA.2KOWOPCVUXWCLN6SVP4LRZUYKBKMP7XDWB7OJHQENFMTVF4JSLFA'
APPEUIs_AM107 = ["q4-1003-7456", "eui-24e124128c147470", "eui-24e124128c147446", "eui-24e124710c408089", "eui-24e124128c147500", "eui-24e124128c147204", "eui-24e124725c461468"]
APPEUIs_AM300 = ["am3019-testqc2090", "am307-9074"]

# Call back functions
def on_message(mqttc, userdata, msg):
    hour = datetime.now(pytz.timezone('Europe/Madrid'))
    hour_values = hour.strftime("%Y-%m-%d %H:%M:%S")

    print("\nMessage received, ", hour_values)

    if msg.topic.startswith('v3/' + APPID + '/devices/'):
        on_message2(mqttc, userdata, msg)

# Gives connection message
def on_connect(mqttc, mosq, obj, rc):
    print("Connected with result code:" + str(rc))
    # Subscribe to the uplink messages of all devices AM107
    for appeui in APPEUIs_AM107:
        mqttc.subscribe('v3/' + APPID + '/devices/' + appeui + '/up')
        print("Subscribed to topic: v3/" + APPID + "/devices/" + appeui + "/up")

    # Subscribe to the uplink messages of all devices AM300
    for appeui in APPEUIs_AM300:
        mqttc.subscribe('v3/' + APPID + '/devices/' + appeui + '/up')
        print("Subscribed to topic: v3/" + APPID + "/devices/" + appeui + "/up")

sensors_name = {
    "q4-1003-7456": "Q4-1003",
    "eui-24e124128c147470": "UNKNOWN",
    "eui-24e124128c147446": "UNKNOWN",
    "eui-24e124710c408089": "OpenLab - Laser Room",
    "eui-24e124128c147500": "OpenLab - Main Room",
    "eui-24e124128c147204": "DigitalLab",
    "eui-24e124725c461468": "Test AM103 qc2090",
    "am3019-testqc2090": "Test device qc2090",
    "am307-9074": "Computer Room"
}

def on_message2(mqttc, userdata, msg):
    payload = json.loads(msg.payload.decode())

    try:
      decoded_payload = payload["uplink_message"]["decoded_payload"]
      end_device_ids = payload["end_device_ids"]
      device_id = end_device_ids.get("device_id")

      if device_id in sensors_name:
            room_name = sensors_name[device_id]
            print("Sensor Data for", room_name + ":")

            if "uplink_message" in payload and "decoded_payload" in payload["uplink_message"]:
                decoded_payload = payload["uplink_message"]["decoded_payload"]

                if device_id in APPEUIs_AM107:
                  #AM107
                  activity = decoded_payload.get("activity")
                  co2 = decoded_payload.get("co2")
                  humidity = decoded_payload.get("humidity")
                  illumination = decoded_payload.get("illumination")
                  infrared = decoded_payload.get("infrared")
                  infrared_and_visible = decoded_payload.get("infrared_and_visible")
                  pressure = decoded_payload.get("pressure")
                  temperature = decoded_payload.get("temperature")
                  tvoc = decoded_payload.get("tvoc")

                  print("Activity:", activity)
                  print("CO2:", co2)
                  print("Humidity:", humidity)
                  print("Illumination:", illumination)
                  print("Infrared:", infrared)
                  print("Infrared and Visible:", infrared_and_visible)
                  print("Pressure:", pressure)
                  print("Temperature:", temperature)
                  print("TVOC:", tvoc)

                  collected_data.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), room_name, co2, humidity, illumination, temperature, tvoc])
                  # Save data to Excel after printing sensor data
                  save_data_to_excel(collected_data, "sensor_data.xlsx")
                  #post_data(collected_data)
                elif device_id in APPEUIs_AM300:
                  #AM300
                  co2 = decoded_payload.get("co2")
                  humidity = decoded_payload.get("humidity")
                  light_level = decoded_payload.get("light_level")
                  o3 = decoded_payload.get("o3")
                  pir = decoded_payload.get("pir")
                  pm10 = decoded_payload.get("pm10")
                  pm2_5 = decoded_payload.get("pm2_5")
                  pressure = decoded_payload.get("pressure")
                  temperature = decoded_payload.get("temperature")
                  tvoc = decoded_payload.get("tvoc")

                  print("CO2:", co2)
                  print("Humidity:", humidity)
                  print("Light Level:", light_level)
                  print("O3:", o3)
                  print("PIR:", pir)
                  print("PM10:", pm10)
                  print("PM2.5:", pm2_5)
                  print("Pressure:", pressure)
                  print("Temperature:", temperature)
                  print("TVOC:", tvoc)

                  collected_data.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), room_name, co2, humidity, light_level, temperature, tvoc])
                  # Save data to Excel after printing sensor data
                  save_data_to_excel(collected_data, "sensor_data.xlsx")
                  #post_data(collected_data)
                else:
                  print("Unknown device type:", device_id)
            else:
              print("Error: Incomplete payload")
      else:
          print("Error: Unknown device ID")
    except Exception as e:
      print("Error:", e)

mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.username_pw_set(APPID, PSW)
mqttc.connect("eu1.cloud.thethings.network", 1883, 60)

mqttc.loop_forever()


Connected with result code:0
Subscribed to topic: v3/sensors-openlab@ttn/devices/q4-1003-7456/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/eui-24e124128c147470/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/eui-24e124128c147446/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/eui-24e124710c408089/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/eui-24e124128c147500/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/eui-24e124128c147204/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/eui-24e124725c461468/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/am3019-testqc2090/up
Subscribed to topic: v3/sensors-openlab@ttn/devices/am307-9074/up

Message received,  2024-05-03 11:01:43
Sensor Data for Test device qc2090:
CO2: 559
Humidity: 34
Light Level: 3
O3: 0.06
PIR: idle
PM10: 6
PM2.5: 6
Pressure: 1004.5
Temperature: 23.6
TVOC: 100
Data saved to 'sensor_data.xlsx' successfully.

Message received,  2024-05-03 11:03:18
Sensor Data for Q4-1003:
Ac